### 1. Bibliotheken importieren und Arbeitsumgebung einrichten

Zuerst werden die notwendigen Bibliotheken importiert:  
  
`pandas`: Biblikothek für die Datenmanipulation  
`lxml` bzw. `etree`: für das Parsen von XML-Daten  
`unicodedata`: Für die Arbeit mit Unicode-Zeichen und -Strings  
`plotly.express`: leistungsstarke Bibliothek in Python für die Erstellung von interaktiven Visualisierungen und Diagrammen  


In [ ]:
import pandas as pd
from lxml import etree
import unicodedata
import plotly.express as px

### 2. XML-Datei direkt als XML laden

Um die Anzahl der gefundenen Datensätze auszugeben, werden diese mittels etree als XML geladen.

In [ ]:
# Laden der MARC-xml-Datei in ElementTree: 
tree = etree.parse('Mittelalter.xml')

# Laden des Root-Verzeichnisses des XML:
root = tree.getroot()                

# Definiere den Namensraum für MARC21 - muss bei Bedarf angepasst werden (wenn bspw. ein anderes Metadatenformat genutzt wird):
ns = {'marc': 'http://www.loc.gov/MARC21/slim'} 
# Einlesen der einzelnen enthaltenen Datensätze (records) in eine Liste: 
records = root.findall('.//marc:record', namespaces=ns)

#Ausgabe der Länge der Liste als Kontrolle für die Anzahl der enthaltenen Datensätze:
print("Gefundene Records:", len(records))

### 3. Direkter Zugriff auf die Inhalte des XML

Im Folgenden wird beispielhaft einer der Datensätze aus der Variable "records" separat gespeichert. Daraufhin wird er in einen String umgewandelt, um ihn betrachten zu können. Im Anschluss wird aus diesem Datensatz das "Controlfield" mit dem Tag 001 extrahiert und dessen Inhalt exemplarisch ausgegeben.


In [ ]:
# Beispielhaft einen der Datensätze aus der Liste als eigene Variable speichern:
test_record = records[0]

# In einen String umwandeln: 
test_record_string = etree.tostring(test_record, encoding="utf-8")
print(test_record_string)

Extrahieren des Inhalts des Controlfields mit dem Tag "001" aus "test_record":

In [ ]:
# Angabe des Namespaces: 
ns = {'marc': 'http://www.loc.gov/MARC21/slim'}

# Extraktion des Elements "Controlfield" mit dem Tag "001"
controlfield_001 = test_record.find("marc:controlfield[@tag='001']", namespaces=ns)
print(controlfield_001)

In [ ]:
# Ausgabe des Inhalts des Controlfields: 
print(controlfield_001.text)

### 4. Funktion zur Extraktion mehrerer Inhalte aus den einzelnen Datensätzen:

Die Funktion `parse_record` erwartet nun immer einen einzelnen Datensatz (record), den sie verarbeitet. Hierzu ist zunächst wieder der passende Namespace defininiert.

Zusätzlich wurde eine weitere Funktion `extract_text` definiert. Diese erwartet eine Pfadangabe zu einem XML-Element wie bspw. `"marc:controlfield[@tag='001']"` oder auch `"marc:datafield[@tag='245']/marc:subfield[@code='a']"`. Die Funktion sucht dann nach den entsprechenden Elementen im XML und extrahiert den Inhalt. Falls mehrere Felder mit demselben Pfad enthalten sind, werden die Inhalte durch "; " zu einem String verbunden.

Anschließend wird für jeden gewünschten Inhalt eine Variable definiert (bspw. "titel"), die dann auf die Funktion `extract_text`

Nun werden die einzelnen Datensätze aus der Menge der gesammelten Datensätze in der Variable records nacheinander (Datensatz für Datensatz) der Funktion parse_record übergeben und die zurückgewonnenen Inhalte in der Variable result gespeichert:
 zugreift, dieser den angegebenen Pfad übergibt und den zurückgegebenen Inhalt speichert.

Zuletzt werden die Inhalte jeder erstellten Variable einem passenden Schlüssel zugeordnet und als einfaches Dictionary (Sammlung von Schlüssel-Werte-Paaren) zurückgegeben.

In [ ]:
# Funktion zum Extrahieren von Datensätzen
def parse_record(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    
    def extract_text(xpath_query):
        fields = record.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")
    title = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='a']")
    subtitle = extract_text("marc:datafield[@tag='245']/marc:subfield[@code='b']")
    creator = extract_text("marc:datafield[@tag='100']/marc:subfield[@code='a']")
    place = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='a']")
    publisher = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='b']")
    year = extract_text("marc:datafield[@tag='264']/marc:subfield[@code='c']")

    return {
        "idn": idn,
        "creator": creator,
        "title": title,
        "subtitle": subtitle,
        "publisher": publisher,
        "place": place,
        "year": year        
    }

Nun werden die einzelnen Datensätze aus der Menge der gesammelten Datensätze in der Variable `records` nacheinander (Datensatz für Datensatz) der Funktion `parse_record` übergeben und die zurückgewonnenen Inhalte in der Variable `result` gespeichert:


In [ ]:
# Übergabe der einzelnen Datensätze an die Funktion "parse_record":
result = [parse_record(record) for record in records]

In [ ]:
print(result)

Abschließend können die Inhalte aus der Variable `result` in eine Tabelle in Form eines Pandas-Dataframes für die weitere Arbeit umgewandelt werden:

In [ ]:
# Umwandlung in ein Pandas-Dataframe
df = pd.DataFrame(result)
df#.head()

### 5. Speichern des Dataframes

Dataframes können in mehreren Formaten gespeichert werden:

Speichern als CSV:

In [ ]:
df.to_csv("Publikationsdaten.csv", encoding="utf-8")

Speichern als Excel-Tabelle:

In [ ]:
df.to_excel("Publikationsdaten.xlsx")

Speichern als Pandas-Dataframe für die weitere Nutzung in Python:

In [ ]:
df.to_hdf("Publikationsdaten.h5", key="df")